In [2]:
import os
import pandas as pd
import numpy as np
from progressbar import progressbar

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 400)
pd.options.display.float_format = '{:5}'.format
# pd.options.display.max_rows = None

In [4]:
def process_box_scores(dir: str="../data/nba_boxscores", to_file=False) -> None:
    '''Calculate and generate box scores of games
    Params:
        dir: directory of basic master, generates if none
    '''
    if dir is None:
        dir = "../data/nba_boxscores"
    
    lu = pd.read_csv("../data/lu/nba_team_lu.csv")
    lu.drop(["NBA_team_name"], axis=1, inplace=True)
    lu.dropna(inplace=True)

    team_stats_list = []
    broken_games = ['0022200065.csv', '00222000617.csv']

    for filename in progressbar(os.listdir(dir)):
        f = os.path.join(dir, filename)

        # checking if it is a file
        if not os.path.isfile(f):
            continue

        if os.path.split(filename)[1] in broken_games:
            continue

    
        df_game = pd.read_csv(f)

        game_id = df_game["GAME_ID"][0]

        df_game.drop(columns=['Unnamed: 0', 'GAME_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'MIN', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'FG_PCT', 'FG3_PCT', 'FT_PCT',], inplace=True, errors="ignore")
        
        team1_id = df_game["TEAM_ID"][0]
        team2_id = df_game["TEAM_ID"].iloc[-1]
        
        t1_stats = df_game.loc[df_game["TEAM_ID"] == team1_id].drop(columns=["TEAM_ID"])
        t2_stats = df_game.loc[df_game["TEAM_ID"] == team2_id].drop(columns=["TEAM_ID"])


        t1_sum_stats = pd.concat((pd.DataFrame([[game_id, team1_id]], columns=["GAME_ID", "TEAM_ID"]),t1_stats.sum(axis=0).to_frame().T), axis=1)
        t2_sum_stats = pd.concat((pd.DataFrame([[game_id, team2_id]], columns=["GAME_ID", "TEAM_ID"]), t2_stats.sum(axis=0).to_frame().T), axis=1)

        t1_sum_stats["FG_PCT"] = t1_sum_stats["FGM"] / t1_sum_stats["FGA"]
        t1_sum_stats["FG3_PCT"] = t1_sum_stats["FG3M"] / t1_sum_stats["FG3A"]
        t1_sum_stats["FT_PCT"] = t1_sum_stats["FTM"] / t1_sum_stats["FTA"]

        t2_sum_stats["FG_PCT"] = t2_sum_stats["FGM"] / t2_sum_stats["FGA"]
        t2_sum_stats["FG3_PCT"] = t2_sum_stats["FG3M"] / t2_sum_stats["FG3A"]
        t2_sum_stats["FT_PCT"] = t2_sum_stats["FTM"] / t2_sum_stats["FTA"]

        t1_sum_stats['SCORE_DIFF'] = t1_sum_stats["PTS"] - t2_sum_stats["PTS"]
        t2_sum_stats['SCORE_DIFF'] = t2_sum_stats["PTS"] - t1_sum_stats["PTS"]

        t1_sum_stats["WON"] = 1 if t1_sum_stats['SCORE_DIFF'][0] > 0 else 0
        t2_sum_stats["WON"] = 1 if t2_sum_stats['SCORE_DIFF'][0] > 0 else 0
        
        team_stats_list.append(t1_sum_stats)
        team_stats_list.append(t2_sum_stats)

    team_stats = pd.concat(team_stats_list)

    game_stats = pd.merge(team_stats, team_stats, on="GAME_ID", suffixes=["_T1", "_T2"])
    game_stats.drop(game_stats.loc[game_stats["TEAM_ID_T1"] == game_stats["TEAM_ID_T2"]].index, inplace=True)

    game_stats.drop_duplicates(subset=['GAME_ID'], inplace=True)
    game_stats.drop(game_stats.loc[(~ game_stats['TEAM_ID_T1'].isin(lu['team_id'])) | (~ game_stats['TEAM_ID_T2'].isin(lu['team_id']))].index, inplace=True)

    if to_file:
        team_stats.to_csv("../data_proc/team_games_stats.csv", index=False)
        game_stats.to_csv("../data_proc/games_stats.csv", index=False)

process_box_scores("../data/nba_boxscores", to_file=True)
    

100% (15365 of 15365) |##################| Elapsed Time: 0:00:56 Time:  0:00:56


In [5]:
from helpers.csv_helpers import combine_csv_dir

def game_id_df(dir: str="../data/nba_gamelogs/", update_master=False, to_file=False) -> pd.DataFrame:
    '''Mapping df between NBA Game_ID (ex. 21600885) and betting (ex. "20231111_nba_Miami_Atlanta") IDs 
    
    Params:
        dir: directory of nba gamelogs
        join_files: create/use processed combined gamelogs
        reload_master: update processed combined gamelogs
        to_file: write to file
    '''
    lu = pd.read_csv("../data/lu/nba_team_lu.csv")
    lu.drop(["NBA_team_name", "team_id"], axis=1, inplace=True)
    lu.dropna(inplace=True)
    abbrv_to_sbr = lu.set_index("NBA_team_abbrev").T.to_dict('records')[0]

        
    df = combine_csv_dir(dir)

    df.drop_duplicates(["TEAM_ID", "GAME_ID"], inplace=True)
    game_id_df = df[["GAME_ID", "GAME_DATE", "MATCHUP"]]
    game_id_df["GAME_DATE"] = game_id_df["GAME_DATE"].apply(lambda x: x.split("T")[0].replace("-", ""))
    game_id_df[["TEAM1", "TEAM2"]] = game_id_df["MATCHUP"].str.split(r' .* ', expand=True)
    game_id_df["TEAM1"] = game_id_df["TEAM1"].map(abbrv_to_sbr)
    game_id_df["TEAM2"] = game_id_df["TEAM2"].map(abbrv_to_sbr)

    game_id_df["SBR_GAME_ID"] = game_id_df["GAME_DATE"] + "_nba_" + game_id_df["TEAM1"] + "_" + game_id_df["TEAM2"]
    game_id_df = game_id_df[["GAME_ID", "SBR_GAME_ID"]]

    if to_file:
        game_id_df.to_csv("game_id_proc.csv", index=False)
    return game_id_df


game_id_df("../data/nba_gamelogs/", to_file=True, update_master=True)

100% (12 of 12) |########################| Elapsed Time: 0:00:01 Time:  0:00:01
/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_54703/2341679076.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_id_df["GAME_DATE"] = game_id_df["GAME_DATE"].apply(lambda x: x.split("T")[0].replace("-", ""))
/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_54703/2341679076.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_id_df[["TEAM1", "TEAM2"]] = game_id_df["MATCHUP"].str.split(r' .* ', 

,GAME_ID,SBR_GAME_ID
0,11800001,NaN
9,11800001,NaN
26,11800002,20180928_nba_Charlotte_Boston
28,11800002,20180928_nba_Boston_Charlotte
53,11800003,20180929_nba_Toronto_Portland
...,...,...
30539,41400404,20150611_nba_Golden State_Cleveland
30558,41400405,20150614_nba_Cleveland_Golden State
30560,41400405,20150614_nba_Golden State_Cleveland
30575,41400406,20150616_nba_Cleveland_Golden State


In [19]:
def games_basic_master(boxscore_dir:str="../data/nba_boxscores", 
                        gamelogs_dir:str="../data/nba_gamelogs/", 
                        bets_dir:str="../data/nba_lines_historical",
                        to_file=False) -> pd.DataFrame:
    '''Generates master games dataframe with game id's, final scores, and betting lines

    Params:
        boxscore_dir: directory for boxscores, runs process_box_scores on directory if no csv detected
        gamelogs_dir: directory for gamelogs, runs game_id_df on directory if no csv detected
        bets_dir: directory for betting lines, combines with boxscore and gamelogs
    '''

    if not os.path.isfile("final_scores.csv"):
        games_stats = process_box_scores(boxscore_dir)[1]
    else:
        games_stats = pd.read_csv("games_stats.csv")

    if not os.path.isfile("game_id_proc.csv"):
        game_id_df = game_id_df(gamelogs_dir)
    else:
        game_id_df = pd.read_csv("game_id_proc.csv")
    
    master_df = pd.DataFrame()

    # process bets

    # moneylines -> probabilities
    
    f = os.path.join(bets_dir, f"nba_historical_moneyline.csv")

    df = pd.read_csv(f)
    df.dropna(inplace=True)
    df["away_prob"] = df["away_line"].apply(lambda x: -100 / x if x < 0 else x / 100)
    df["away_prob"] = 1 - df["away_prob"] / (df["away_prob"] + 1)
    df["home_prob"] = df["home_line"].apply(lambda x: -100 / x if x < 0 else x / 100)
    df["home_prob"] = 1 - df["home_prob"] / (df["home_prob"] + 1)
    df.drop(df[(df["away_prob"] + df["home_prob"] >= 1.4)].index, inplace=True) # 1.3 is profit margin for sports bettings
    df.drop(["sport", "bet_type", "away", "home"], axis=1, inplace=True)

    bets_master = df

    # spread
    f = os.path.join(bets_dir, f"nba_historical_spread.csv")

    df = pd.read_csv(f)
    df.dropna(inplace=True)
    df.drop(df[(df["away_line"] != -df["home_line"])].index, inplace=True) # home line opposite of away line
    df.drop(["sport", "date", "bet_type", "away", "home", "home_line"], axis=1, inplace=True)
    df.rename(columns={"away_line": "away_spread"}, inplace=True)
    bets_master = bets_master.merge(df, on="game_id", how="outer")

    # total
    f = os.path.join(bets_dir, f"nba_historical_total.csv")

    df = pd.read_csv(f)
    df.dropna(inplace=True)
    df.drop(df[(df["away_line"] != df["home_line"])].index, inplace=True) # home line equal away line
    df.drop(["sport", "date", "bet_type", "away", "home", "home_line"], axis=1, inplace=True)
    df.rename(columns={"away_line": "away_total"}, inplace=True)
    bets_master = bets_master.merge(df, on="game_id", how="outer")

    bets_master.dropna(subset=["away_prob", "away_spread", "away_total"], inplace=True)
    
    # final scores
    master_df = games_stats

    # game ids
    master_df = master_df.merge(game_id_df, on="GAME_ID")
    master_df.insert(1, "SBR_GAME_ID", master_df.pop("SBR_GAME_ID"))
    
    # betting lines
    master_df = master_df.merge(bets_master, left_on="SBR_GAME_ID", right_on="game_id")
    master_df.drop("game_id", axis=1, inplace=True)
    master_df.insert(2, "DATE", master_df.pop("date"))

    if to_file:
        master_df.to_csv("games_basic_master.csv", index=False)

    return master_df
    

games_basic_master(to_file=True)

,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,...,FG3M_T2,FG3A_T2,FTM_T2,FTA_T2,OREB_T2,DREB_T2,REB_T2,AST_T2,STL_T2,BLK_T2,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_line,home_line,away_prob,home_prob,away_spread,away_total
0,21600885,20170227_nba_Atlanta_Boston,20170227.0,1610612737,46.0,95.0,6.0,25.0,16.0,20.0,13.0,42.0,55.0,22.0,10.0,10.0,14.0,19.0,114.0,80.0,0.4842105263157895,0.24,0.8,16.0,1,...,10.0,34.0,20.0,25.0,9.0,31.0,40.0,21.0,9.0,7.0,18.0,20.0,98.0,-80.0,0.3908045977011494,0.2941176470588235,0.8,-16.0,0,155.0,-180.0,0.392156862745098,0.6428571428571428,4.5,212.5
1,21900278,20191129_nba_Chicago_Portland,20191129.0,1610612741,36.0,94.0,14.0,38.0,17.0,25.0,11.0,36.0,47.0,24.0,8.0,2.0,11.0,22.0,103.0,-20.0,0.3829787234042553,0.3684210526315789,0.68,-4.0,0,...,10.0,30.0,21.0,28.0,8.0,41.0,49.0,20.0,6.0,11.0,13.0,21.0,107.0,20.0,0.4523809523809524,0.3333333333333333,0.75,4.0,1,260.0,-320.0,0.2777777777777778,0.7619047619047619,7.0,227.0
2,41900313,20200922_nba_L.A. Lakers_Denver,20200922.0,1610612747,43.0,83.0,6.0,26.0,14.0,22.0,4.0,21.0,25.0,27.0,11.0,3.0,16.0,26.0,106.0,-40.0,0.5180722891566265,0.2307692307692307,0.6363636363636364,-8.0,0,...,11.0,29.0,23.0,29.0,9.0,35.0,44.0,26.0,8.0,2.0,18.0,21.0,114.0,40.0,0.547945205479452,0.3793103448275862,0.7931034482758621,8.0,1,-234.0,178.0,0.7005988023952097,0.35971223021582743,-6.0,211.5
3,22100624,20220112_nba_Cleveland_Utah,20220112.0,1610612739,43.0,81.0,11.0,30.0,14.0,17.0,7.0,43.0,50.0,28.0,6.0,3.0,13.0,17.0,111.0,100.0,0.5308641975308642,0.3666666666666666,0.8235294117647058,20.0,1,...,16.0,47.0,7.0,11.0,4.0,28.0,32.0,11.0,7.0,2.0,7.0,16.0,91.0,-100.0,0.4,0.3404255319148936,0.6363636363636364,-20.0,0,175.0,-210.0,0.36363636363636365,0.6774193548387097,5.5,222.5
4,22000156,20210111_nba_Indiana_Sacramento,20210111.0,1610612754,47.0,97.0,13.0,39.0,15.0,18.0,10.0,27.0,37.0,26.0,14.0,3.0,12.0,21.0,122.0,-25.0,0.4845360824742268,0.3333333333333333,0.8333333333333334,-5.0,0,...,15.0,32.0,18.0,23.0,7.0,37.0,44.0,29.0,6.0,9.0,18.0,17.0,127.0,25.0,0.5595238095238095,0.46875,0.782608695652174,5.0,1,-165.0,143.0,0.6226415094339622,0.41152263374485587,-3.5,229.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,21800513,20181227_nba_New York_Milwaukee,20181227.0,1610612752,36.0,97.0,13.0,31.0,11.0,16.0,10.0,33.0,43.0,21.0,8.0,6.0,10.0,20.0,96.0,-80.0,0.3711340206185567,0.4193548387096774,0.6875,-16.0,0,...,11.0,32.0,17.0,21.0,11.0,46.0,57.0,20.0,4.0,11.0,15.0,14.0,112.0,80.0,0.4516129032258064,0.34375,0.8095238095238095,16.0,1,837.0,-1400.0,0.10672358591248665,0.9333333333333333,14.0,226.5
9644,21700136,20171105_nba_Atlanta_Cleveland,20171105.0,1610612737,41.0,86.0,11.0,25.0,24.0,34.0,11.0,33.0,44.0,24.0,4.0,6.0,17.0,20.0,117.0,10.0,0.4767441860465116,0.44,0.7058823529411765,2.0,1,...,10.0,36.0,21.0,23.0,15.0,32.0,47.0,27.0,6.0,6.0,13.0,29.0,115.0,-10.0,0.4375,0.2777777777777778,0.9130434782608696,-2.0,0,614.0,-900.0,0.14005602240896364,0.9,11.0,220.0
9645,21400830,20150223_nba_Philadelphia_Miami,20150223.0,1610612755,40.0,84.0,13.0,31.0,15.0,28.0,9.0,24.0,33.0,27.0,10.0,8.0,12.0,18.0,108.0,-55.0,0.4761904761904761,0.4193548387096774,0.5357142857142857,-11.0,0,...,11.0,21.0,16.0,22.0,11.0,34.0,45.0,27.0,6.0,3.0,16.0,20.0,119.0,55.0,0.5542168674698795,0.5238095238095238,0.7272727272727273,11.0,1,575.0,-800.0,0.14814814814814814,0.8888888888888888,11.5,193.5
9646,21600644,20170120_nba_Sacramento_Memphis,20170120.0,1610612758,31.0,76.0,6.0,29.0,23.0,32.0,5.0,30.0,35.0,18.0,9.0,6.0,10.0,21.0,91.0,-80.0,0.4078947368421052,0.2068965517241379,0.71875,-16.0,0,...,11.0,28.0,20.0,25.0,15.0,43.0,58.0,21.0,6.0,4.0,15.0,24.0,

In [20]:
def game_wl_records(dir:str=None, to_file=False) -> pd.DataFrame:
    '''Generate dataframe of games with previous win/loss of each games
    
    Params:
        dir: directory of basic master, generates if none
    '''
    if dir is None:
        dir = "games_basic_master.csv"
    
    lu = pd.read_csv("../data/lu/nba_team_lu.csv")
    lu.drop(["NBA_team_name"], axis=1, inplace=True)
    lu.dropna(inplace=True)

    df = pd.read_csv(dir).sort_values(by="DATE")
    result_df = df.copy()
    result_df[["10wp_T1", "25wp_T1", "50wp_T1", "75wp_T1", "100wp_T1", "10wp_T2", "25wp_T2", "50wp_T2", "75wp_T2", "100wp_T2"]] = np.nan

    for team_id in progressbar(lu["team_id"].unique()):
        # team_id = lu["team_id"].unique()[1]
        team_df = df.copy().loc[(df["TEAM_ID_T1"] == team_id) | (df["TEAM_ID_T2"] == team_id)]
        team_df.loc[df["TEAM_ID_T1"] == team_id, "TEAM_WON"] = team_df["WON_T1"]
        team_df.loc[df["TEAM_ID_T2"] == team_id, "TEAM_WON"] = team_df["WON_T2"]

        team_df["10wp"] = team_df["TEAM_WON"].rolling(10, min_periods=10).mean().shift(1)
        team_df["25wp"] = team_df["TEAM_WON"].rolling(25, min_periods=25).mean().shift(1)
        team_df["50wp"] = team_df["TEAM_WON"].rolling(50, min_periods=50).mean().shift(1)
        team_df["75wp"] = team_df["TEAM_WON"].rolling(75, min_periods=75).mean().shift(1)
        team_df["100wp"] = team_df["TEAM_WON"].rolling(100, min_periods=100).mean().shift(1)

        
        team_df = team_df[["GAME_ID", "10wp", "25wp", "50wp", "75wp", "100wp"]]
        team_df.insert(1, "TEAM_ID", team_id)

        team1_df = team_df.rename(columns={c: f"{c}_T1" for c in team_df.columns if c not in ['GAME_ID']})
        team2_df = team_df.rename(columns={c: f"{c}_T2" for c in team_df.columns if c not in ['GAME_ID']})

        team1_df = df.merge(team1_df, how="left", on=["GAME_ID", "TEAM_ID_T1"])
        team2_df = df.merge(team2_df, how="left", on=["GAME_ID", "TEAM_ID_T2"])

        result_df.update(team1_df)
        result_df.update(team2_df)

    result_df.sort_values(by="DATE", inplace=True)

    if to_file:
        result_df.to_csv("games_wp_master.csv", index=False)

    return result_df

game_wl_records(to_file=True)

100% (30 of 30) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,...,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_line,home_line,away_prob,home_prob,away_spread,away_total,10wp_T1,25wp_T1,50wp_T1,75wp_T1,100wp_T1,10wp_T2,25wp_T2,50wp_T2,75wp_T2,100wp_T2
1,21400003,20141028_nba_Houston_L.A. Lakers,20141028.0,1610612745,31.0,73.0,12.0,29.0,34.0,50.0,14.0,33.0,47.0,22.0,7.0,3.0,13.0,30.0,108.0,90.0,0.4246575342465753,0.4137931034482758,0.68,18.0,1,...,11.0,32.0,90.0,-90.0,0.3544303797468354,0.3,0.7948717948717948,-18.0,0,-300.0,250.0,0.75,0.2857142857142857,-7.0,207.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,21400001,20141028_nba_Orlando_New Orleans,20141028.0,1610612753,32.0,84.0,4.0,11.0,16.0,21.0,16.0,40.0,56.0,17.0,5.0,9.0,18.0,25.0,84.0,-85.0,0.3809523809523809,0.3636363636363636,0.7619047619047619,-17.0,0,...,9.0,17.0,101.0,85.0,0.4059405940594059,0.2352941176470588,0.4838709677419355,17.0,1,380.0,-475.0,0.2083333333333333,0.8260869565217391,9.0,191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21400002,20141028_nba_Dallas_San Antonio,20141028.0,1610612742,38.0,78.0,8.0,21.0,16.0,19.0,9.0,24.0,33.0,17.0,9.0,3.0,10.0,20.0,100.0,-5.0,0.4871794871794871,0.3809523809523809,0.8421052631578947,-1.0,0,...,20.0,20.0,101.0,5.0,0.5285714285714286,0.5,0.8125,1.0,1,140.0,-165.0,0.4166666666666666,0.6226415094339622,3.5,203.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,21400013,20141029_nba_L.A. Lakers_Phoenix,20141029.0,1610612747,37.0,86.0,4.0,13.0,21.0,25.0,12.0,26.0,38.0,20.0,9.0,3.0,13.0,18.0,99.0,-100.0,0.4302325581395348,0.3076923076923077,0.84,-20.0,0,...,14.0,25.0,119.0,100.0,0.5487804878048781,0.5,0.7222222222222222,20.0,1,750.0,-1100.0,0.1176470588235294,0.9166666666666666,12.0,208.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21400015,20141029_nba_Oklahoma City_Portland,20141029.0,1610612760,33.0,81.0,2.0,16.0,21.0,26.0,14.0,29.0,43.0,19.0,9.0,5.0,13.0,28.0,89.0,-85.0,0.4074074074074074,0.125,0.8076923076923077,-17.0,0,...,10.0,19.0,106.0,85.0,0.4482758620689655,0.3793103448275862,0.7727272727272727,17.0,1,380.0,-475.0,0.2083333333333333,0.8260869565217391,9.0,198.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,22300008,20231110_nba_Philadelphia_Detroit,20231110.0,1610612755,35.0,83.0,9.0,28.0,35.0,41.0,11.0,35.0,46.0,23.0,13.0,5.0,8.0,23.0,114.0,40.0,0.4216867469879518,0.3214285714285714,0.8536585365853658,8.0,1,...,16.0,26.0,106.0,-40.0,0.4651162790697674,0.2857142857142857,0.8181818181818182,-8.0,0,-387.5,302.5,0.7948717948717949,0.2484472049689442,-8.5,224.5,0.7,0.72,0.68,0.72,0.67,0.2,0.16,0.16,0.21333333333333335,0.22
9643,22300010,20231110_nba_Brooklyn_Boston,20231110.0,1610612751,38.0,85.0,15.0,37.0,16.0,18.0,9.0,37.0,46.0,26.0,0.0,3.0,10.0,25.0,107.0,-70.0,0.4470588235294118,0.4054054054054054,0.8888888888888888,-14.0,0,...,7.0,13.0,121.0,70.0,0.4361702127659574,0.3653846153846153,0.7142857142857143,14.0,1,472.5,-674.895031490553,0.1746724890829694,0.8709502630212449,12.0,229.25,0.4,0.4,0.44,0.5333333333333333,0.51,0.7,0.56,0.62,0.6533333333333333,0.67
9641,22300014,20231110_nba_L.A. Clippers_Dallas,20231110.0,1610612746,43.0,88.0,8.0,27.0,32.0,36.0,19.0,28.0,47.0,21.0,7.0,2.0,14.0,27.0,126.0,-90.0,0.4886363636363636,0.2962962962962963,0.8888888888888888,-18.0,0,...,8.0,23.0,144.0,90.0,0.5204081632653061,0.4523809523809524,0.8846153846153846,18.0,1,-103.498950021,-116.497700091996,0.5085969731554854,0.5381013287554225,1.0,236.0,0.3,0.48,0.52,0.49333333333333335,0.53,0.6,0.44,0.42,0.4666666666666667,0.49
9642,22300015,20231110_nba_L.A. Lakers_Phoenix,20231110.0,1610612747,42.0,84.0,12.0,27.0,26.0,34.0,9.0,34.0,43.0,27.0,9.0,7.0,11.

In [21]:
def game_rolling_stats(dir:str=None, to_file=False) -> pd.DataFrame:
    '''Generate dataframe of games with previous win/loss of each games
    
    Params:
        dir: directory of basic master, generates if none
    '''
    if dir is None:
        dir = "games_basic_master.csv"
    
    lu = pd.read_csv("../data/lu/nba_team_lu.csv")
    lu.drop(["NBA_team_name"], axis=1, inplace=True)
    lu.dropna(inplace=True)

    intervals = [10, 25, 50, 75, 100]

    df = pd.read_csv(dir).sort_values(by="DATE")

    stats_cols = pd.unique(pd.Series([c[:-3] for c in df.columns if c not in ['GAME_ID', 'SBR_GAME_ID', 'DATE', 'TEAM_ID_T1', 'TEAM_ID_T2', 'away_line', 'home_line', 'away_prob', 'home_prob', 'away_spread', 'away_total']]))

    result_df = df.copy()
    result_df = pd.concat([result_df, pd.DataFrame(columns=[f'{i}g_{c}_{t}' for i in intervals for c in stats_cols for t in ['T1', 'T2']])])

    for team_id in progressbar(lu["team_id"].unique()):
        # team_id = lu["team_id"].unique()[2]
        team_df = df.copy().loc[(df["TEAM_ID_T1"] == team_id) | (df["TEAM_ID_T2"] == team_id)]

        s_frame = pd.DataFrame()
        for s in stats_cols:
            team_df.loc[df["TEAM_ID_T1"] == team_id, s] = team_df[f'{s}_T1']
            team_df.loc[df["TEAM_ID_T2"] == team_id, s] = team_df[f'{s}_T2']

            i_frame = pd.DataFrame()
            for i in intervals:
                i_frame[f'{i}g_{s}'] = team_df[s].rolling(i, min_periods=i).mean().shift(1)
            
            s_frame = pd.concat([s_frame, i_frame], axis=1)
        
        team_df = pd.concat([team_df, s_frame], axis=1)


        team_df = team_df[["GAME_ID"]+[f'{i}g_{c}' for i in intervals for c in stats_cols]]
        team_df.insert(1, "TEAM_ID", team_id)


        team1_df = team_df.rename(columns={c: f"{c}_T1" for c in team_df.columns if c not in ['GAME_ID']})
        team2_df = team_df.rename(columns={c: f"{c}_T2" for c in team_df.columns if c not in ['GAME_ID']})

        team1_df = df.merge(team1_df, how="left", on=["GAME_ID", "TEAM_ID_T1"])
        team2_df = df.merge(team2_df, how="left", on=["GAME_ID", "TEAM_ID_T2"])

        result_df.update(team1_df)
        result_df.update(team2_df)

    result_df.sort_values(by="DATE", inplace=True)

    if to_file:
        result_df.to_csv("games_rolling_master.csv", index=False)

    return result_df

game_rolling_stats(to_file=True)

  0% (0 of 30) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

KeyError: 'home_l_T1'

In [11]:
len(pd.read_csv("games_rolling_master.csv").columns)


262